# 1. Traitement des résultats du Kmeans

**17 août 2021** : j'ai modifié l'ensemble de ce qui vient de dataiku. Normalement, cela devrait fonctionner partout. Des erreurs peuvent rester néanmoins, soyez vigilant.

**11 mai 2021** : on utilise la version 2 de la base finale, donc la version 3 des modèles.

**Qu'est-ce qui change ?** Après la version 2 (en date du 08/05/21), on généralise encore la fonction qui construit l'indicatrice sommaire : on parcourt les différents mots du début du sommaire, on lâche un peu de mou dans la sélection du sommaire, on sélectionne brutalement les 200 lignes après l'apparition du mot débutant le sommaire si besoin etc... Bon, même la détection de l'apparition est sujette à caution : parfois, les indices semblent lointains. C'est pourquoi on a rajouter une restriction. Si l'indice du sommaire (c'est-à-dire le numéro de la ligne où débuterait le sommaire) est trop élevé (et que l'on a toujours pas de sommaire pour l'étude), on ne prend pas de sommaire et on accepte (pour le moment) de laisser cette étude vide finalement. Cette restriction est basée sur l'indice moyen des études ayant un sommaire correct en bout de pipeline, qui est de 44, pour être moins rigide nous avons fixé ce seuil à 100. De même parfois, les indicatrices sommaires sont trop petites, donc si en première approche il y a moins de 50 lignes indiquées, on poursuit avec la nouvelle approche.
En soi, on pourrait justement dire que le sommaire se trouve probablement dans le 400 premières lignes (par exemple) et ainsi prendre encore plus brutalement ces lignes, mais cela reste à discuter.

In [7]:
import pandas as pd, numpy as np
from bs4 import BeautifulSoup
import time
from unidecode import unidecode
import re
import pickle
import os

# Vous pouvez indiquer votre chemin ici
# os.chdir("C:\\Users\\theo.roudil-valentin\\Documents\\Codes")

# Sinon lancez simplement, cela reprendre vos config si vous avez déjà fait tourner les codes précédents
cwd=os.getcwd()
cwd
if cwd.split('\\')[-1] not in ['Bagging_model','Extraction_sommaire']:
    try:
        os.chdir("2021-NLP_AE\\Data\\Bagging_model")
    except:
        os.chdir("2021-NLP_AE\\Pipeline\\Extraction_sommaire")

base_finale=pickle.load(open("/base_pour_Bagging_final_new.pickle",'rb'))
print('Il y a ',len(np.unique(base_finale.num_etude)),"études")
print(base_finale.label_k.sum()/len(base_finale)*100)

changement


In [20]:
300000/len(base_finale)*100
base_finale.columns

12.279812724669407

In [19]:
len(base_finale[base_finale.ind_sommaire==1][base_finale.label_k==1])/len(base_finale)*100

c:\Users\theo.roudil-valentin\venv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


1.1597055137177787

In [10]:
len(base_finale[base_finale.label_k==0])

515683
Index(['phrase', 'num_etude', 'phrase_2', '0', 'ventre_0', 'ventre_1',
       'ventre_2', 'ventre_3', 'ventre_4', 'ventre_5', 'ventre_6', 'ventre_7',
       'ventre_8', 'ventre_9', 'ventre_10', 'ventre_11', '33', '34', '35',
       '36', '37', '38', '39', 'f_mots', 'f_carac', 'html_carac',
       'ind_sommaire', 'sommaire_longueur', 'sommaire_int', 'caractere_spec',
       'label_k_4'],
      dtype='object')

Attention à bien vérifier la correspondance des labels du Kmeans !!

## 1.1 Vérification des labelisés 1 par le Kmeans

**ATTENTION** Pour la version 3, les labels Kmeans sont inversés !

**Si vous ré-adaptez ce code : faites attention à quel groupe le Kmeans a labellisé comme 0 et 1 !**

In [ ]:
N=10000

for i in base_finale[base_finale.label_k_4==1].iloc[:N,:].index:
    print("Indice :",i,"\n",base_finale.phrase_2[i])

Bon sur l'échantillon de 1000, les labelisés 1 semblent être tous des "non-titres", donc c'est super !

## 1.2 Vérification des labelisés 0 par le Kmeans
### 1.2.1 Essai sur sous-ensemble

In [ ]:
N=0#10000

for i in base_finale[base_finale.ind_sommaire==1][base_finale.label_k==0].iloc[N:N+10000,:].index:
    print("Indice :",i,"\n",base_finale.phrase_2[i])

Bon sur l'échantillon de 1000, les labelisés 1 semblent être tous des "non-titres", donc c'est super !

## 1.2 Vérification des labelisés 0 par le Kmeans
### 1.2.1 Essai sur sous-ensemble

In [ ]:
N=0#10000

for i in base_finale[base_finale.ind_sommaire==1][base_finale.label_k==0].iloc[N:N+10000,:].index:
    print("Indice :",i,"\n",base_finale.phrase_2[i])

**ATTENTION** La cellule qui suit a pour but de nettoyer les mauvaises labellisations effectuées par le Kmeans. Elle concerne donc NOTRE projet, très probablement pas le votre. Donc faîtes attention quand vous le lancez.

Comme vous pouvez le constater, on essaie ici de chercher les index des unités étranges, pour forcer leur label à 0.

In [108]:
base=base_finale.iloc[:296183,:]
c=[1 if (base.ind_sommaire[i]==1) and (base.label_k[i]==1) else 0 for i in base.index]
#print(len(c))
c_index=[i for i in base.index if base.phrase_2[i]=='^' or base.phrase_2[i]==base.iloc[141175,:].phrase_2]
for i in c_index:
    c[i]=0
#print(len(c))
c[29533:29584]=np.zeros(29584-29533)
#print(len(c))
c[29901:131462]=np.zeros(131462-29901)
#print(len(c))
c[141223:162733]=np.zeros(162733-141223)
#print(len(c))
#c[162550:162535]=np.zeros(162535-162550)
c[177975:186252]=np.zeros(186252-177975)
#print(len(c))
c[203924:247583]=np.zeros(247583-203924)
#print(len(c))
c[284870:285811]=np.zeros(285811-284870)
#print(len(c))
c[285860:296183]=np.zeros(296183-285860)
#print(len(c))
base['label']=c
base=base_finale.iloc[:296183,:]
c=[1 if (base.ind_sommaire[i]==1) and (base.label_k[i]==1) else 0 for i in base.index]
#print(len(c))
c_index=[i for i in base.index if base.phrase_2[i]=='^' or base.phrase_2[i]==base.iloc[141175,:].phrase_2]
for i in c_index:
    c[i]=0
#print(len(c))
c[29533:29584]=np.zeros(29584-29533)
#print(len(c))
c[29901:131462]=np.zeros(131462-29901)
#print(len(c))
c[141223:162733]=np.zeros(162733-141223)
#print(len(c))
#c[162550:162535]=np.zeros(162535-162550)
c[177975:186252]=np.zeros(186252-177975)
#print(len(c))
c[203924:247583]=np.zeros(247583-203924)
#print(len(c))
c[284870:285811]=np.zeros(285811-284870)
#print(len(c))
c[285860:296183]=np.zeros(296183-285860)
#print(len(c))
base['label']=c
base

<ipython-input-5-391cfff0ded8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['label']=c


,phrase,num_etude,phrase_2,0,ventre_0,ventre_1,ventre_2,ventre_3,ventre_4,ventre_5,...,39,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec,label_k_4,label
0,<title>EI Munchhouse_V3</title>,1003691,EI Munchhouse_V3,0.000000,0.007237,0.001262,0.209389,0.139448,0.353363,0.130169,...,0.0,0.002841,0.006463,0.005215,0.0,0.0,0.0,0.0,0,0.0
1,"<h1><a name=""bookmark0""></a><span class=""font2...",1003691,06,0.048998,0.021633,0.027711,0.201385,0.156727,0.325818,0.098187,...,0.0,0.000000,0.000431,0.027724,0.0,0.0,0.0,0.0,0,0.0
2,"<p><span class=""font20"" style=""font-weight:bol...",1003691,2019,0.022272,0.011876,0.013931,0.204308,0.128357,0.321930,0.092321,...,0.0,0.000000,0.001293,0.014548,0.0,0.0,0.0,0.0,0,0.0
3,"<p><span class=""font14"">Agence Aquitaine</span...",1003691,Agence Aquitaine,0.001114,0.011479,0.006782,0.209288,0.135008,0.264427,0.141187,...,0.0,0.002841,0.006463,0.010705,0.0,0.0,0.0,0.0,0,0.0
4,"<p><span class=""font14"">SAFEGE</span></p>",1003691,SAFEGE,0.001114,0.007209,0.006764,0.206694,0.135066,0.262446,0.141680,...,0.0,0.000000,0.002154,0.007960,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296178,"<li>\n<p><span class=""font6"">■ Les aléas d’acc...",106905,\n■ Les aléas d’accumulation de gaz dangereux ;,0.001114,0.026082,0.008910,0.202207,0.138018,0.272561,0.147820,...,0.0,0.019886,0.019388,0.021136,1.0,0.0,0.0,1.0,0,0.0
296179,"<li>\n<p><span class=""font6"">■ Les aléas d’ino...",106905,\n■ Les aléas d’inondation localisée ou de rem...,0.001114,0.051407,0.008833,0.197512,0.134242,0.262987,0.145136,...,0.0,0.048295,0.044808,0.037332,1.0,0.0,0.0,1.0,0,0.0
296180,"<p><a name=""bookmark921""></a><span class=""font...",106905,Sur le territoire d’étude,0.001114,0.031688,0.027998,0.204303,0.149004,0.247218,0.107150,...,0.0,0.008523,0.010340,0.034312,1.0,0.0,0.0,0.0,0,0.0
296181,"<p><span class=""font6"">Le territoire de La Jai...",106905,Le territoire de La Jaille-Yvon n’est pas cons...,0.001114,0.060613,0.006379,0.202552,0.128435,0.253922,0.137603,...,0.0,0.056818,0.056010,0.041998,1.0,0.0,0.0,0.0,0,0.0


In [ ]:
#c_index
#base[base.phrase_2==base.iloc[141175,:].phrase_2]
base.label.sum()/base.shape[0]*100

In [ ]:
for i in base[base.label==1].index:
    print("Indice :",i,"\n","base.phrase_2[i]")

 Désormais, on a une nouvelle variable : label.
"Elle est le croisement de indicatrice_sommaire et label_k puis passe dans un nouveau nettoyage. On peut donc donner les anciennes variables utilisées pour construire notre nouvel algorithme."


# 2. Classification supervisée

Attention, désormais, titres ==1 et non_titres==0

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(base.iloc[:,3:-1],base.loc[:,'label'],test_size=0.25)


In [ ]:
print("Train : proportion de 1 :",round(y_train.sum()/len(y_train)*100,3)," ; de 0 :",round((1-y_train.sum()/len(y_train))*100,3))
print("Test : proportion de 1 :",round(y_test.sum()/len(y_test)*100,3)," ; de 0 :",round((1-y_test.sum()/len(y_test))*100,3))


### Arbre de décision

In [ ]:
k=0 # On a plus besoin d'étudier le k
X_train_2=X_train.iloc[:,:(X_train.shape[1]-k)]
X_test_2=X_test.iloc[:,:(X_train.shape[1]-k)]

from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(class_weight="balanced")
DTC.fit(X_train_2, y_train)
print("Score train :",round(DTC.score(X_train_2,y_train),3))
print("Score test :",round(DTC.score(X_test_2,y_test),3))


from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,DTC.predict(X_test_2))

### Forêt aléatoire

In [ ]:
################ NE PAS LANCER !\n",
######### De même, plus besoin d'étudier cela, je laisse pour l'historique du travail\n",
absi=[]
ordo=[]
ordo_test=[]

for i in range(1,X_train.shape[1]):
    X_train_2=X_train.iloc[:,:i]
    X_test_2=X_test.iloc[:,:i]

    DTC = RandomForestClassifier(n_estimators=10,verbose=0,class_weight="balanced")
    DTC.fit(X_train_2, y_train)

    ordo.append(DTC.score(X_train_2,y_train))
    ordo_test.append(DTC.score(X_test_2,y_test))
    absi.append(i)

import matplotlib.pyplot as plt
f,a=plt.subplots(1,figsize=(12,6))
a.plot(absi,ordo)
a.plot(absi,ordo_test)
a.set(xlabel="Nombre de variables",ylabel='Score'
      title='Score de la random forest en fonction du nombre de variables')
plt.legend(['train','test'])

In [ ]:
############## On passe aux forêts :\n",
n=10 # On essaie avec 10 estimateurs\n",

from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=n,verbose=0,class_weight="balanced")
RFC.fit(X_train, y_train)

print("Score train :",round(RFC.score(X_train,y_train),3))
print("Score test :",round(RFC.score(X_test,y_test),3))

from sklearn.metrics import confusion_matrix
print("Matrice de confusion sur le train \n",confusion_matrix(y_train,RFC.predict(X_train)))
print("Matrice de confusion sur le test \n",confusion_matrix(y_test,RFC.predict(X_test)))


On fait encore des erreurs, donc on poursuit, on augmente n

In [ ]:
n=100 # On essaie avec 100 estimateurs

from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=n,verbose=0,class_weight="balanced")
RFC.fit(X_train, y_train)

print("Score train :",round(RFC.score(X_train,y_train),3))
print("Score test :",round(RFC.score(X_test,y_test),3))

from sklearn.metrics import confusion_matrix
print("Matrice de confusion sur le train \n",confusion_matrix(y_train,RFC.predict(X_train)))
print("Matrice de confusion sur le test \n",confusion_matrix(y_test,RFC.predict(X_test)))


"Malgré la multiplication par 10 du nombre d'estimateurs, on ne réussit pas à classifier parfaitement. Poursuivons."
### Forêt aléatoire et bagging
On va faire 100 fois la sélection du meilleur estimateur parmi 100 forêt aléatoire estimée.

In [ ]:
from sklearn.ensemble import BaggingClassifier

RFC = RandomForestClassifier(n_estimators=100,verbose=0,class_weight="balanced")#,bootstrap=False) #à voir
Bagging = BaggingClassifier(RFC,n_estimators=100,verbose=0)
Bagging.fit(X_train, y_train)

In [ ]:
print("Score train :",round(Bagging.score(X_train,y_train),3))
print("Score test :",round(Bagging.score(X_test,y_test),3))

from sklearn.metrics import confusion_matrix
print("Matrice de confusion sur le train \n",confusion_matrix(y_train,Bagging.predict(X_train)))
print("Matrice de confusion sur le test \n",confusion_matrix(y_test,Bagging.predict(X_test)))

Malgré le Bagging, on ne réussit pas à classer **parfaitement**.

In [ ]:
base['label_rf']=Bagging.predict(base.iloc[:,3:-1])

In [ ]:
for i in base[base.label_rf==1].phrase_2:
        print(i)

In [ ]:
(base.label_rf.sum()/len(base))*100

In [ ]:
import pickle
dico_Bagging={}
dico_Bagging['train']=[X_train,y_train]
dico_Bagging['test']=[X_test,y_test]
dico_Bagging['base']=base
dico_Bagging['modele']=Bagging
pickle.dump(dico_Bagging,open("dico_Bagging_new.pickle",'wb'))

# 3. Prédiction supervisée de l'ensemble de notre base

In [ ]:
########### Chargement du modèle\n",
import pandas as pd, numpy as np
import time
import pickle
dico=pickle.load(open("dico_Bagging_new.pickle",'rb'))
clf_titres_fin=dico['modele']

In [ ]:
base_finale=pickle.load(open("base_pour_Bagging_final_new.pickle",'rb'))

Attention à bien laisser la dernière variable de la grosse base puisque le modèle utilise label_k pour prédire label

In [ ]:
start=time.time()
pred=Bagging.predict(base_finale.iloc[:,3:])
end=time.time()
print("Durée de la prédiction :",round((end-start)/60,2),"minutes")

In [ ]:
print(pred.shape,base_finale.shape)
base_finale['label_RF']=pred
base_finale.label_RF.sum()/len(base_finale)*100

In [ ]:
for i in base_finale[base_finale.label_RF==1].iloc[:,:].phrase_2:
    print(i)

In [ ]:
pickle.dump(base_finale.loc[:,['num_etude','phrase_2','label_RF']],open("label_RF_new.pickle",'wb'))

# 4. Active learning

Ici, on va essayer de partir de l'aval du processus, pour remonter et modifier les étapes délicates. Ces changements ont déjà été intégrés plus haut, donc je le laisse pour information.

## 4.1 Première boucle

In [ ]:
N=300000
train_active=base_finale.iloc[base.shape[0]:base.shape[0]+N,:]

In [ ]:
for i in train_active.index:
    if train_active.label_RF[i]==1:
        print("\nIndice :",i-base.shape[0])
        print(train_active.loc[i,'phrase_2'])

In [ ]:
c=train_active.label_RF.values
c[2107:26521]=np.zeros(26521-2107)
c[277263:278418]=np.zeros(278418-277263)
c[278867:280569]=np.zeros(280569-278867)
c[280603:294368]=np.zeros(294368-280603)
train_active['label']=c
train_active

In [ ]:
vide=[i for i in np.unique(train_active.num_etude) if train_active[train_active.num_etude==i].label_RF.sum()==0]
vide

In [ ]:
#for i in vide:\n",
i=vide[-1]
print("########################################## \n",i,"\n")
for k in train_active[train_active.num_etude==i].phrase_2:
    print(list(train_active[train_active.num_etude==i].phrase_2).index(k))
    print(k,"\n")

In [ ]:
c=np.zeros(len(train_active[train_active.num_etude==i].phrase_2))
c[18:624]=np.ones(624-18)
idx=train_active[train_active.num_etude==i].index[0]
train_active.loc[18+idx:624+idx,'label']=1
train_active[train_active.num_etude==i][18:100]

In [ ]:
presque_vide=[i for i in np.unique(train_active.num_etude) if (train_active[train_active.num_etude==i].label_RF.sum()>0) and (train_active[train_active.num_etude==i].label_RF.sum()<50)]
presque_vide

In [ ]:
i=presque_vide[-1]
print("########################################## \n",i,"\n")
for k in train_active[train_active.num_etude==i].phrase_2:
    print(list(train_active[train_active.num_etude==i].phrase_2).index(k))
    print(k,"\n")

In [ ]:
idx=train_active[train_active.num_etude==109735].index[0]
train_active.loc[36+idx:257+idx,'label']=1
train_active[train_active.num_etude==109735][36:100]

In [ ]:
idx=train_active[train_active.num_etude==1094735].index[0]
train_active.loc[53+idx:551+idx,'label']=1
train_active.loc[53+idx:551+idx,:]

In [ ]:
base_active=pd.concat([base.iloc[:,:-2],train_active.iloc[:,:-2]])
y_active=list(base.loc[:,'label_rf'])+list(train_active.loc[:,'label'])

In [ ]:
base_active['label']=y_active

In [ ]:
for i in base_active.index:
    try:
        if type(float(base_active.loc[i,'phrase_2']))==float:
            print(base_active.loc[i,'phrase_2'])
            base_active.loc[i,'label']=0
    except:
        continue

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(base_active.iloc[:,3:],base_active.iloc[:,-1],test_size=0.25,shuffle=True)

In [ ]:
n=100 # On essaie avec 100 estimateurs\n",
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=n,verbose=0,class_weight="balanced")
RFC.fit(X_train, y_train)

print("Score train :",round(RFC.score(X_train,y_train),3))
print("Score test :",round(RFC.score(X_test,y_test),3))

from sklearn.metrics import confusion_matrix
print("Matrice de confusion sur le train \n",confusion_matrix(y_train,RFC.predict(X_train)))
print("Matrice de confusion sur le test \n",confusion_matrix(y_test,RFC.predict(X_test)))


In [ ]:
dico_Bagging={}
dico_Bagging['train']=[X_train,y_train]
dico_Bagging['test']=[X_test,y_test]
dico_Bagging['base']=base_active.iloc[:,:3]
dico_Bagging['modele']=RFC
pickle.dump(dico_Bagging,open("dico_RF_active.pickle",'wb'))

#### On check les nouveaux résultats

base_finale

In [ ]:
pred_active=RFC.predict(base_finale.iloc[:,3:-1])
base_finale['label_active']=pred_active

In [ ]:
for i in base_finale[base_finale.label_active==1].phrase_2:
    print(i)

In [ ]:
vide=[i for i in np.unique(base_finale.num_etude) if base_finale[base_finale.num_etude==i].label_active.sum()==0]
len(vide)

In [ ]:
train_active[train_active.num_etude==1098190][50:150]

In [ ]:
vide
#112862
#196750
#1008101
#1078725
#1259375
#2072251 c'est un avis ???
#2255947

# 5. Analyse des titres non détectés (exploratoire, à ne pas faire tourner sauf exception)

Cette partie concerne uniquement notre projet. Elle peut être intéressante pour le votre, mais cela est à vous de voir.

Dans un premier temps, quantifions la quantité de documents n'ayant pas de titres détectés.

In [ ]:
vide=[i for i in np.unique(base_finale.num_etude) if base_finale[base_finale.num_etude==i].label_RF.sum()==0]

In [ ]:
np.unique(base_finale.num_etude)[0]

In [ ]:
non_vide=[i for i in np.unique(base_finale.num_etude) if i not in vide]
len(non_vide)/len(np.unique(base_finale.num_etude))

In [ ]:
index_vide=[True if base_finale.num_etude[i] in vide else False for i in base_finale.index]
base_relou=base_finale[index_vide]
base_relou

In [ ]:
print("Proportion des labels kmeans chez les vides,\nnon-titres (1) :",round(base_relou.label_k.sum()/len(base_relou)*100,2))
print("Proportion des labels kmeans chez les non vides,\nnon-titres (1) :",round(base_propre.label_k.sum()/len(base_propre)*100,2))
print("Proportion des labels kmeans total,\nnon-titres (1) :",round(base_finale.label_k.sum()/len(base_finale)*100,2))
numero=np.unique(base_relou.num_etude)
k=0
titres=list(base_relou[base_relou.label_k==1][base_relou.num_etude==numero[k]].phrase_2.values)
for i in titres:
    print('\nIndice :',titres.index(i),i)

In [ ]:
numero=np.unique(base_relou.num_etude)
k=0
titres=list(base_relou[base_relou.ind_sommaire==1][base_relou.num_etude==numero[k]].phrase_2.values)
for i in titres:
    print('\\nIndice :',titres.index(i),i)

In [ ]:
base_relou[base_relou.ind_sommaire==1][base_relou.num_etude==numero[k]]

Problème sur le clustering Kmeans, probablement du à ind_sommaire ?


In [ ]:
len(base_relou),len(base_finale)

In [ ]:
index_nonvide=[bool(abs(i-1)) for i in index_vide]
base_propre=base_finale[index_nonvide]
distrib_nbligne_propre=[len(base_propre[base_propre.num_etude==i]) for i in np.unique(base_propre.num_etude)]
print("Pourcentage d'études vides :",round(len(non_vide)/len(np.unique(base_finale.num_etude))*100,2),"%")
print('Les documents vides représentent',round(len(base_relou)/len(base_finale)*100,2),'% de la base pour la classification supervisée')

In [ ]:
numero=np.unique(base_propre.num_etude)
k=8
titres=list(base_propre[base_propre.label_k==1][base_propre.num_etude==numero[k]].phrase_2.values)
for i in titres:
    print('\nIndice :',titres.index(i),i)

In [ ]:
distrib_nbligne_vide=[len(base_relou[base_relou.num_etude==i]) for i in np.unique(base_relou.num_etude)]

In [ ]:
def stat_des(x,mo,k=2):
    if mo=='mean':
        y=np.mean(x)
    elif mo=='std':
        y=np.std(x)
    elif mo=='med':
        y=np.median(x)
    else:
        raise ValueError('Veuillez spécifier quel type de stat des vous voulez. Valeurs possibles : mean, std ou med')
    z=round(y,k)
    return z

In [ ]:
import matplotlib.pyplot as plt
figue,hache=plt.subplots(figsize=(18,12))
#sns.kdeplot(distrib_nbligne_propre , bw = 0.5 , fill = True)
#sns.kdeplot(distrib_nbligne_vide , bw = 0.5 , fill = True)
sns.distplot(distrib_nbligne_propre)
sns.distplot(distrib_nbligne_vide)

#hache.hist(distrib_nbligne_propre,density=True,bins=20)
#hache.hist(distrib_nbligne_vide,density=True,bins=20)
plt.legend(['propre','vide'])

Maintenant qu'on a analysé le problème et trouvé une source de problème, on repasse la base_relou à la moulinette pour voir si ça s'arrange

In [ ]:
technique=['non technique',unidecode('Table des matières'.lower())]
numero=np.unique(base_relou.num_etude)
indice=[i for i in range(len(base_relou[base_relou.num_etude==numero[3]].phrase_2.values))
    if np.sum([1 if som in unidecode(base_relou[base_relou.num_etude==numero[3]].phrase_2.values[i].lower()) else 0 for som in technique])>0]


In [ ]:
numero=np.unique(base_relou.num_etude)
print("Il y a ",len(numero),"études vides")
k=8

titres=list(base_relou[base_relou.ind_sommaire==1][base_relou.num_etude==numero[k]].phrase_2.values)\n",
for i in titres:
    print('\nIndice :',titres.index(i),"\n",i)

In [ ]:
numero=np.unique(base_relou.num_etude)
index=[i for i in numero if np.sum(base_relou[base_relou.num_etude==i].ind_sommaire)==0]
base_relou_2=base_relou[[True if base_relou.num_etude[i] in index else False for i in base_relou.index]]
print("Les documents vides tenaces représentent :",round(len(base_relou_2)/len(base_relou)*100,2),'% des lignes de la base relou')
print("On a réussi à traiter ",round((1-len(np.unique(base_relou_2.num_etude))/len(numero))*100,2),"% des documents vides initiaux")
print("Il y a ",len(np.unique(base_relou_2.num_etude)),"études vides")"


In [ ]:
numero_2=np.unique(base_relou_2.num_etude)
k=0

titres=list(base_relou_2[base_relou_2.label_k==0][base_relou_2.num_etude==112862].phrase_2.values)
for i in titres:
    print('\nIndice :',titres.index(i),"\n",i)

In [ ]:
liste=base_relou_2[base_relou_2.label_k==1][base_relou_2.num_etude==101872].phrase_2.values
s=sommaire(liste)
print(s)
#ind_titres(base_relou_2)
a=light_cleaning(liste[s[0]+1]) #hypothèse donc que le 1er titre vient après le sommaire
print(a)
print(''.join([i for i in a]))
j=indice_relou(liste,s,0.4,0.4)
j

In [ ]:
ouais=ind_titres(base_relou_2)
c=functools.reduce(
    operator.iconcat,ouais,[])
print(len(c))
base_relou_2['ind_sommaire']=c

In [ ]:
numero=np.unique(base_relou_2.num_etude)
print("Il y a ",len(numero),"études vides")
k=9

titres=list(base_relou_2[base_relou_2.ind_sommaire==1][base_relou_2.num_etude==numero[k]].phrase_2.values)
for i in titres:
    print('\nIndice :',titres.index(i),"\n",i)

In [ ]:
numero=np.unique(base_relou_2.num_etude)
index=[i for i in numero if np.sum(base_relou_2[base_relou_2.num_etude==i].ind_sommaire)==0]
base_relou_3=base_relou_2[[True if base_relou_2.num_etude[i] in index else False for i in base_relou_2.index]]
print("Les documents vides tenaces représentent :",round(len(base_relou_3)/len(base_relou_2)*100,2),'% des lignes de la base relou')
print("On a réussi à traiter ",round((1-len(np.unique(base_relou_3.num_etude))/len(numero))*100,2),"% des documents vides initiaux")
print("Il y a ",len(np.unique(base_relou_3.num_etude)),"études vides")

In [ ]:
def raffinement_sommaire(base_3,a,b):
    base=base_3.copy()
    c=[1 if len(i)>a or len(i)<b else 0 for i in base.phrase_2]#np.zeros(len(base))
    c_=np.zeros(len(base))
    for i in range(len(base)):
        try:
            k=int(base.phrase_2[i])
            #print(k)\n",
            c_[i]=1
        except:
            continue
    base['sommaire_longueur']=c
    base['sommaire_int']=c_
    return base
base_=raffinement_sommaire(base_relou,180,3)
base_.sommaire_int=base_.sommaire_int*base_.ind_sommaire
base_.sommaire_longueur=base_.sommaire_longueur*base_.ind_sommaire
base_

In [ ]:
kmeans=pickle.load(open("kmeans_model_new.pickle",'rb'))
label_k_relou=kmeans.predict(base_.iloc[:,3:-2])
base_['label_k']=label_k_relou

In [ ]:
label_RF_relou=Bagging.predict(base_.iloc[:,3:-1])
base_['label_RF']=label_RF_relou

In [ ]:
numero=np.unique(base_.num_etude)
k=2
titres=list(base_[base_.label_RF==1][base_.num_etude==numero[k]].phrase_2.values)
for i in titres:
    print('\nIndice :',titres.index(i),i)

In [ ]:
import seaborn as sns
sns.histplot(distrib_nbligne_propre,stat='frequency',bins=100)
